In [ ]:
# imports
from sit1m_data_preprocessing import *
import numpy as np

In [2]:
# download (if not downloaded) and build sift1m dataset
dataset_download_path = "D:/College/SysML/dataset" # a folder to cache dataset contents that are downloaded
splits = build_sift1m(dataset_download_path)

Building Sift1M


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:04<00:00,  4.84s/ url]
Extraction completed...: 0 file [00:04, ? file/s]
Dl Completed...: 100%|██████████| 1/1 [00:04<00:00,  4.84s/ url]


In [3]:
# make and print train split input array (1 million embedding arrays of length 128)
train_input_array = get_train_split(splits)

print(train_input_array)
print(train_input_array.shape[1])

Building train split array
Found existing train data file
[[  0.  16.  35. ...  25.  23.   1.]
 [ 14.  35.  19. ...  11.  21.  33.]
 [  0.   1.   5. ...   4.  23.  10.]
 ...
 [ 30.  12.  12. ...  50.  10.   0.]
 [  0.   5.  12. ...   1.   2.  13.]
 [114.  31.   0. ...  25.  16.   0.]]
128


In [4]:
# make and print test split input array (10k embedding arrays of length 128)
test_input_array, neighbors = get_test_split(splits)

print(test_input_array)
print(test_input_array.shape[1])

Building test split array
Progress: 50.0%
Progress: 100.0%
[[  1.   3.  11. ...  42.  48.  11.]
 [ 40.  25.  11. ...   3.  19.  13.]
 [ 28.   4.   3. ...   2.  54.  47.]
 ...
 [  0.  15.  64. ...   3.  62. 118.]
 [131.   2.   0. ...   7.   0.   0.]
 [ 23.   0.   0. ...  79.  16.   4.]]
128


## PQ Test

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import faiss

In [ ]:
dim = train_input_array.shape[1]
numSubVectors = 8
subVectorBits = 8

pq = faiss.IndexPQ(dim, numSubVectors, subVectorBits)

In [ ]:
pq.train(train_input_array)

In [ ]:
pq.add(train_input_array)

In [ ]:
k = 5
distances, indexes = pq.search(test_input_array[:10000], k)

In [ ]:
t = 25
i = 0
relevantPercent = []

for entry in neighbors:

    topIndexs = []
    for x in range(t):
        index_dict = entry[x]
        topIndexs.append(index_dict["index"])

    search_results = indexes[i]

    count = 0

    for x in search_results:
        if x in topIndexs:
            count+=1

    relevantPercent.append(count/k)
    
    i=i+1
        

print(sum(relevantPercent) / len(relevantPercent))

0.5851


## Non-FSQ VQVAE 

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import numpy as np

class VQVAE(nn.Module):
    def __init__(self, input_dim, codebook_size, codebook_dim):
        super(VQVAE, self).__init__()
        
        self.input_dim = input_dim
        self.codebook_size = codebook_size
        self.codebook_dim = codebook_dim
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, codebook_dim)
        )
        
        # Codebook
        self.codebook = nn.Embedding(codebook_size, codebook_dim)
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(codebook_dim, 256),
            nn.ReLU(),
            nn.Linear(256, input_dim)
        )

    def encode(self, x):
        z = self.encoder(x)
        return z

    def quantize(self, z):
        # Find nearest embedding in the codebook
        z = z.unsqueeze(2)
        distances = torch.norm(z - self.codebook.weight.unsqueeze(0), dim=1)
        indices = torch.argmin(distances, dim=1)
        quantized = self.codebook(indices)
        return quantized, indices

    def decode(self, z):
        x_recon = self.decoder(z)
        return x_recon

    def forward(self, x):
        z = self.encode(x)
        quantized, indices = self.quantize(z)
        x_recon = self.decode(quantized)
        return x_recon, indices

In [6]:
vqvae = VQVAE(input_dim=128, codebook_size=512, codebook_dim=64)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(vqvae.parameters(), lr=0.001)

In [7]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        # You may need to preprocess the data here, such as converting to tensor
        return torch.tensor(sample, dtype=torch.float32)

In [ ]:
train_dataset = CustomDataset(train_input_array)
print(len(train_dataset))

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
lr = 3
seed = 44
EPOCHS = 25
BATCH_SIZE = 64
D = train_input_array.shape[1]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


torch.random.manual_seed(seed)
model = VQVAE(input_dim=128, codebook_size=512, codebook_dim=64)

model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

loss = nn.MSELoss()

log_loss = np.array([])
for i_epoch in range(EPOCHS):
    dataloader = DataLoader(train_input_array, batch_size=BATCH_SIZE, shuffle=True)
    for i_batch, batch in enumerate(dataloader):
        x = batch.to(device)
        x = x.to(torch.float)
        x = x.reshape(BATCH_SIZE, 1, D)

        optimizer.zero_grad()
        out, indices = model(x)

        rec_loss = loss(out, x)
        rec_loss.backward()
        optimizer.step()
        print(f"Epoch: {i_epoch}, Batch: {i_batch}, Loss: {rec_loss}")
        log_loss = np.append(log_loss,np.log(rec_loss.detach().cpu().numpy()))

        torch.save({
            'epoch': i_epoch,
            'model_state_dict':  model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'log_loss': log_loss,
        }, f"vqvae_vimeo_checkpoint.pth")

plt.plot(log_loss)
plt.savefig("vqvae_vimeo_loss.png")

In [ ]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(vqvae.parameters(), lr=0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

vqvae.to(device)

# Training loop
epochs = 25
for epoch in range(epochs):
    vqvae.train()
    total_recon_loss = 0.0
    total_commitment_loss = 0.0
    
    for batch_idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        
        # Forward pass
        inputs = data.to(device)
        recon_batch, indices = vqvae(inputs)
        
        # Compute loss
        recon_loss = criterion(recon_batch, inputs)
        commitment_loss = torch.mean(torch.norm((indices.float() - vqvae.codebook(indices)) ** 2))
        loss = recon_loss + commitment_loss
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Accumulate loss
        total_recon_loss += recon_loss.item()
        total_commitment_loss += commitment_loss.item()
        
    # Print epoch statistics
    print(f"Epoch [{epoch+1}/{epochs}], Recon Loss: {total_recon_loss / len(train_loader):.4f}, Commitment Loss: {total_commitment_loss / len(train_loader):.4f}")

print("Training finished!")

## Learned Compression Model

In [8]:
import math
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import tensorflow as tf

from compressai.models import CompressionModel
from compressai.models.utils import conv, deconv
from compressai.entropy_models import EntropyBottleneck
from compressor_compressai import Network

import torch.nn.functional as F
from torch import Tensor
from compressai.ops.parametrizers import NonNegativeParametrizer

import zlib


In [10]:
class ChannelToSpatial(nn.Module):
    def forward(self, x):
        batch_size, channels, spatial_dim = x.size()
        return x.view(batch_size, spatial_dim, channels)


class Network(CompressionModel):
    def __init__(self, compressed_d = 64, uncompressed_d = 128):
        """
        compressed_d = compressed dimension of the embedding
        """
        super().__init__()
        self.entropy_bottleneck = EntropyBottleneck(1)
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(32, compressed_d, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(compressed_d, compressed_d, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Conv1d(compressed_d, compressed_d, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(compressed_d, compressed_d//2, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(compressed_d//2, 1, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.LazyLinear(compressed_d),
            nn.ReLU(),
            nn.LazyLinear(compressed_d)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(32, compressed_d, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(compressed_d, uncompressed_d, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(uncompressed_d, uncompressed_d, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(uncompressed_d, uncompressed_d, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(uncompressed_d, uncompressed_d//2, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(uncompressed_d//2, 1, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.LazyLinear(uncompressed_d),
            nn.ReLU(),
            nn.LazyLinear(uncompressed_d)
        )
    
    def forward(self, x):
        y = self.encoder(x)
        
        y_hat, y_likelihoods = self.entropy_bottleneck(y)

        x_hat = self.decoder(y_hat)

        return x_hat, y_likelihoods
    
    def get_compressed_embeddings(self, x):
        y = self.encoder(x)
        
        y_hat, y_likelihoods = self.entropy_bottleneck(y)

        return y_hat, y_likelihoods
    
    def decode_compressed_embeddings(self, y_hat):
        
        x_hat = self.decoder(y_hat)

        return x_hat

In [14]:
__all__ = ["GDN_1d", "GDN1_1d"]

#zlib

class GDN_1d(nn.Module):
    r"""Generalized Divisive Normalization layer.

    Introduced in `"Density Modeling of Images Using a Generalized Normalization
    Transformation" <https://arxiv.org/abs/1511.06281>`_,
    by Balle Johannes, Valero Laparra, and Eero P. Simoncelli, (2016).

    .. math::

       y[i] = \frac{x[i]}{\sqrt{\beta[i] + \sum_j(\gamma[j, i] * x[j]^2)}}

    """

    def __init__(
        self,
        in_channels: int,
        inverse: bool = False,
        beta_min: float = 1e-6,
        gamma_init: float = 0.1,
    ):
        super().__init__()

        beta_min = float(beta_min)
        gamma_init = float(gamma_init)
        self.inverse = bool(inverse)

        self.beta_reparam = NonNegativeParametrizer(minimum=beta_min)
        beta = torch.ones(in_channels)
        beta = self.beta_reparam.init(beta)
        self.beta = nn.Parameter(beta)

        self.gamma_reparam = NonNegativeParametrizer()
        gamma = gamma_init * torch.eye(in_channels)
        gamma = self.gamma_reparam.init(gamma)
        self.gamma = nn.Parameter(gamma)

    def forward(self, x: Tensor) -> Tensor:
        _, C, _ = x.size()

        beta = self.beta_reparam(self.beta)
        gamma = self.gamma_reparam(self.gamma)
        gamma = gamma.reshape(C, C, 1)
        norm = F.conv1d(x**2, gamma, beta)

        if self.inverse:
            norm = torch.sqrt(norm)
        else:
            norm = torch.rsqrt(norm)

        out = x * norm

        return out

class GDN1_1d(GDN_1d):
    r"""Simplified GDN layer.

    Introduced in `"Computationally Efficient Neural Image Compression"
    <http://arxiv.org/abs/1912.08771>`_, by Johnston Nick, Elad Eban, Ariel
    Gordon, and Johannes Ballé, (2019).

    .. math::

        y[i] = \frac{x[i]}{\beta[i] + \sum_j(\gamma[j, i] * |x[j]|}

    """

    def forward(self, x: Tensor) -> Tensor:
        _, C, _ = x.size()

        beta = self.beta_reparam(self.beta)
        gamma = self.gamma_reparam(self.gamma)
        gamma = gamma.reshape(C, C, 1)
        norm = F.conv1d(torch.abs(x), gamma, beta)

        if not self.inverse:
            norm = 1.0 / norm

        out = x * norm

        return out

In [15]:
class ChannelToSpatial(nn.Module):
    def forward(self, x):
        batch_size, channels, spatial_dim = x.size()
        return x.view(batch_size, spatial_dim, channels)


class Network(CompressionModel):
    def __init__(self, compressed_d = 64, uncompressed_d = 128):
        """
        compressed_d = compressed dimension of the embedding
        """
        super().__init__()
        self.entropy_bottleneck = EntropyBottleneck(1)
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(32, compressed_d, kernel_size=3, stride=1, padding=1),
            GDN1_1d(compressed_d),
            nn.Conv1d(compressed_d, compressed_d, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Conv1d(compressed_d, compressed_d, kernel_size=3, stride=1, padding=1),
            GDN1_1d(compressed_d),
            nn.ReLU(),
            nn.Conv1d(compressed_d, compressed_d//2, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(compressed_d//2, 1, kernel_size=3, stride=1, padding=1),
            GDN1_1d(1),
            nn.LazyLinear(compressed_d),
            nn.BatchNorm1d(1),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.LazyLinear(compressed_d),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(32, compressed_d, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(compressed_d),
            nn.ConvTranspose1d(compressed_d, uncompressed_d, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(uncompressed_d, uncompressed_d, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(uncompressed_d),
            nn.ConvTranspose1d(uncompressed_d, uncompressed_d, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(uncompressed_d, uncompressed_d//2, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(uncompressed_d//2, uncompressed_d//2, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(uncompressed_d//2, 1, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.LazyLinear(uncompressed_d),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.LazyLinear(uncompressed_d)
        )
    
    def forward(self, x):
        y = self.encoder(x)
        
        y_hat, y_likelihoods = self.entropy_bottleneck(y)

        x_hat = self.decoder(y_hat)

        return x_hat, y_likelihoods
    
    def get_compressed_embeddings(self, x):
        y = self.encoder(x)
        
        y_hat, y_likelihoods = self.entropy_bottleneck(y)

        return y_hat, y_likelihoods
    
    def decode_compressed_embeddings(self, y_hat):
        
        x_hat = self.decoder(y_hat)

        return x_hat



In [ ]:
# Training

# hyperparamters    
seed = 44
EPOCHS = 15
BATCH_SIZE = 64
lmbda = 1 # parameter for bitrate distortion tradeoff

torch.random.manual_seed(seed)
model = Network(64).to("cuda")
model.train()

# parameters
parameters = set(p for n, p in model.named_parameters() if not n.endswith(".quantiles"))
aux_parameters = set(p for n, p in model.named_parameters() if n.endswith(".quantiles"))
optimizer = optim.Adam(parameters, lr=1e-4)
aux_optimizer = optim.Adam(aux_parameters, lr=1e-3)

D = train_input_array.shape[1]

loss_arr = np.array([])
aux_loss_arr = np.array([])
mse_loss_arr = np.array([])
for i_epoch in range(EPOCHS):
    dataloader = DataLoader(train_input_array, batch_size=BATCH_SIZE, shuffle=True)
    for i_batch, batch in enumerate(dataloader):
        x = batch.to("cuda")
        x = x.to(torch.float)
        x = x.reshape(BATCH_SIZE, 1, D)

        optimizer.zero_grad()
        aux_optimizer.zero_grad()

        x_hat, y_likelihoods = model(x)

        # bitrate of the quantized latent
        N, _, L = x.size()
        num_logits = N * L
        bpp_loss = torch.log(y_likelihoods).sum() / (-math.log(2) * num_logits)

        # mean square error
        mse_loss = F.mse_loss(x, x_hat)

        # final loss term
        loss = mse_loss + lmbda * bpp_loss

        loss.backward()
        optimizer.step()
        aux_loss = model.aux_loss()
        aux_optimizer.step()
        print(f"Epoch: {i_epoch}, Batch: {i_batch}, loss: {loss}, aux_loss: {aux_loss}")
        loss_arr = np.append(loss_arr,loss.detach().cpu().numpy())
        aux_loss_arr = np.append(aux_loss_arr,aux_loss.detach().cpu().numpy())
        mse_loss_arr = np.append(mse_loss_arr,mse_loss.detach().cpu().numpy())

        torch.save({
            'epoch': i_epoch,
            'model_state_dict':  model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss_arr': loss_arr,
            'aux_loss_arr': aux_loss_arr,
            'mse_loss_arr': mse_loss_arr
        }, f"compressai_losses.pth")

plt.plot(loss_arr)
plt.plot(aux_loss_arr)
plt.savefig("compressai_losses.png")

In [19]:
# Compressed Embeddings Playground

model = Network(64)

checkpoint = torch.load("compressai_losses_final.pth",map_location="cuda")

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

model.to("cuda")

# for stacking tensors back into numpy array
tensor_list = []

test_dataloader = DataLoader(test_input_array, batch_size=1, shuffle=False)
for i_batch, batch in enumerate(test_dataloader):
        x = batch.to("cuda")
        x = x.to(torch.float)
        x = x.reshape(1, 1, 128)

        #print(x)
        #x_hat, y_likelihoods = model(x)
        #print(x_hat.to(torch.int))

        y_hat, y_likelihoods = model.get_compressed_embeddings(x)

        y_hat = y_hat.reshape(64)

        tensor_list.append(y_hat)

        if i_batch >= 10:
                break


combined_tensor = torch.stack(tensor_list)
compressed_embeddings = combined_tensor.detach().cpu().numpy()
#print(compressed_embeddings)

print(max(compressed_embeddings.flatten()))
print(min(compressed_embeddings.flatten()))


data = compressed_embeddings[0]

# Compress the data using zlib
compressed_data = zlib.compress(data, level=9)

print("Original size:", len(data))
print("Compressed size:", len(compressed_data))

print(data)
print(compressed_data)


133.0
-124.0
Original size: 64
Compressed size: 160
[  52.   31.   -8. -114.  -55.   45.   68.  -68.   48.  -61.   60.  -84.
  -33.   60.  -75.  -67. -105.   30.   68.  -77.   67.  -63.   89.   52.
  -75.   80.   83.  -96.  -69.   85.   85.  -89.   52. -100.   50.   76.
  -51. -100.    5.  -88.   79.  -35.   75.  -17.   85.   53.  -97.   -4.
 -114.  -83.  -32.  109.   52.  -80.  -59.   83.   69.   87.  -94.  -78.
  -88.   58.   61.  -58.]
[120, 218, 45, 143, 33, 14, 194, 64, 16, 69, 191, 64, 114, 28, 14, 208, 86, 33, 16, 21, 72, 14, 80, 129, 64, 32, 17, 40, 66, 122, 128, 134, 16, 66, 86, 53, 4, 65, 72, 53, 162, 29, 133, 70, 162, 57, 0, 18, 201, 219, 204, 138, 151, 55, 243, 51, 187, 179, 43, 149, 133, 244, 203, 37, 13, 210, 199, 164, 5, 76, 200, 246, 17, 234, 12, 175, 241, 10, 95, 240, 40, 213, 13, 222, 193, 11, 190, 185, 207, 31, 99, 134, 55, 184, 195, 101, 154, 11, 184, 133, 158, 186, 134, 107, 225, 116, 230, 51, 79, 60, 197, 7, 152, 153, 247, 33, 147, 238, 248, 76, 54, 198, 77, 124, 

In [20]:
# Run train set back through model to get compressed embeddings train split

compress_batch_size = 64
D = train_input_array.shape[1]

model = Network(64)

checkpoint = torch.load("compressai_losses_final.pth",map_location="cuda")

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

model.to("cuda")

print("Building compressed train split")

# for stacking tensors back into numpy array
tensor_list = []

compress_dataloader = DataLoader(train_input_array, batch_size=compress_batch_size, shuffle=False)
for i_batch, batch in enumerate(compress_dataloader):
        x = batch.to("cuda")
        x = x.to(torch.float)
        x = x.reshape(compress_batch_size, 1, D)

        y_hat, y_likelihoods = model.get_compressed_embeddings(x)

        y_hat = y_hat.to(torch.int)

        for i in range(y_hat.size(0)): 
                train_entry = y_hat[i, 0, :]  
                tensor_list.append(train_entry)  

        if i_batch % 1562 == 0:
                print("Progress: " + str((i_batch/1562)*10) + "%")

print("Stacking tensors and converting to numpy array")

combined_tensor = torch.stack(tensor_list)
compressed_train_embeddings = combined_tensor.detach().cpu().numpy()

compressed_train_split_path = 'learned_compressed_train_split.npy'

np.save(compressed_train_split_path, compressed_train_embeddings)


Building compressed train split
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 60.0%
Progress: 70.0%
Progress: 80.0%
Progress: 90.0%
Progress: 100.0%
Stacking tensors and converting to numpy array


In [21]:
# Run test set through model to get compressed embeddings test split

compress_batch_size = 64
D = test_input_array.shape[1]

model = Network(64)

checkpoint = torch.load("compressai_losses_final.pth",map_location="cuda")

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

model.to("cuda")

print("Building compressed test split")

# for stacking tensors back into numpy array
tensor_list = []

compress_dataloader = DataLoader(test_input_array, batch_size=compress_batch_size, shuffle=False)
for i_batch, batch in enumerate(compress_dataloader):
        x = batch.to("cuda")
        x = x.to(torch.float)

        if i_batch == 156:
            x = x.reshape(16, 1, D)
        else:
            x = x.reshape(compress_batch_size, 1, D)
                
        y_hat, y_likelihoods = model.get_compressed_embeddings(x)

        y_hat = y_hat.to(torch.int) # 16 bit int

        for i in range(y_hat.size(0)): 
            test_entry = y_hat[i, 0, :]  
            tensor_list.append(test_entry) 

        #if i_batch % 1562 == 0:
            #print("Progress: " + str((i_batch/1562)*10) + "%")

print("Stacking tensors and converting to numpy array")

combined_tensor = torch.stack(tensor_list)
compressed_test_embeddings = combined_tensor.detach().cpu().numpy()

compressed_test_split_path = 'learned_compressed_test_split.npy'

np.save(compressed_test_split_path, compressed_test_embeddings)


Building compressed test split
Stacking tensors and converting to numpy array


In [22]:
# Load compressed embedding splits

compressed_train_split_path = 'learned_compressed_train_split.npy'
compressed_test_split_path = 'learned_compressed_test_split.npy'

compressed_train_split_for_hnsw = np.load(compressed_train_split_path)

#print(compressed_train_split_for_hnsw)

#print(compressed_train_split_for_hnsw[0])

#compressed_train_split_for_hnsw.shape[0]

compressed_test_split_for_hnsw = np.load(compressed_test_split_path)




In [23]:
print(min(compressed_test_split_for_hnsw.flatten()))
print(max(compressed_test_split_for_hnsw.flatten()))

-181
160


In [24]:
import faiss

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import random, time

In [25]:
dim = compressed_train_split_for_hnsw.shape[1]

hnsw_indexer = faiss.index_factory(dim, "HNSW")
dim

64

In [26]:
hnsw_indexer.train(compressed_train_split_for_hnsw)

In [27]:
hnsw_indexer.add(compressed_train_split_for_hnsw)

In [26]:
k = 10
distances, index = hnsw_indexer.search(compressed_test_split_for_hnsw[:2], k)
distances

array([[218., 219., 235., 252., 260., 270., 272., 272., 284., 288.],
       [100., 116., 136., 144., 171., 179., 190., 206., 215., 216.]],
      dtype=float32)

In [34]:
# Size

idx_file = open("./temp.index", "w")
idx_file.truncate(0)
idx_file.close()
faiss.write_index(hnsw_indexer, "./temp.index")
file_size = os.path.getsize('./temp.index')
file_size
#528129506 528mb

528129506

In [37]:
# Relevancy - what percent of the K neighbor search results are in the top T correct closest neighbors

k = 5 # number of returned neighbors for each embedding
t = 25 # top X number of closest neighbors

distances, index = hnsw_indexer.search(compressed_test_split_for_hnsw[:10000], k)

relevantPercent = []

i = 0
for search_results in index:
    closest_neighbors = neighbors[i]
    
    topIndexs = []
    for x in range(t):
        index_dict = closest_neighbors[x]
        topIndexs.append(index_dict["index"])

    count = 0
    for x in search_results:
        if x in topIndexs:
            count+=1

    relevantPercent.append(count/k)
    i=i+1
    
print(sum(relevantPercent) / len(relevantPercent))

0.70636


In [49]:
# Recall@1

k = 100 # number of returned neighbors for each embedding

distances, index = hnsw_indexer.search(compressed_test_split_for_hnsw[:10000], k)

count = 0

i = 0
for search_results in index:
    closest_neighbors = neighbors[i]
    
    topIndexs = []
    for x in range(1):
        index_dict = closest_neighbors[x]
        topIndexs.append(index_dict["index"])

    for x in search_results:
        if x in topIndexs:
            count+=1
            break

    i=i+1
    
print(count/10000)

0.8198


In [29]:
# Search Speed

total_time = 0
iterations = 1000

for x in range(iterations):

    k = 25
    index = random.randint(0, 9999)

    input = np.empty((0,64))
    input = np.vstack([input,compressed_test_split_for_hnsw[index]])

    start_time = time.time()
    distances, index = hnsw_indexer.search(input, k)
    end_time = time.time()

    total_time += end_time - start_time


print("avg search time: ", total_time/iterations, " seconds")

avg search time:  0.0002955641746520996  seconds


In [ ]:
# Test Reconstruction Loss

compress_batch_size = 64
D = test_input_array.shape[1]

model = Network(64)

checkpoint = torch.load("compressai_losses_3.pth",map_location="cuda")

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

model.to("cuda")

print("Test Reconstruction Loss:")

loss_arr = np.array([])
mse_loss_arr = np.array([])

lmbda = 1

compress_dataloader = DataLoader(test_input_array, batch_size=compress_batch_size, shuffle=False)
for i_batch, batch in enumerate(compress_dataloader):
        x = batch.to("cuda")
        x = x.to(torch.float)

        if i_batch == 156:
            x = x.reshape(16, 1, D)
        else:
            x = x.reshape(compress_batch_size, 1, D)
                
        x_hat, y_likelihoods = model(x)

        # bitrate of the quantized latent
        N, _, L = x.size()
        num_logits = N * L
        bpp_loss = torch.log(y_likelihoods).sum() / (-math.log(2) * num_logits)

        # mean square error
        mse_loss = F.mse_loss(x, x_hat)

        # final loss term
        loss = mse_loss + lmbda * bpp_loss

        loss_arr = np.append(loss_arr,loss.detach().cpu().numpy())
        mse_loss_arr = np.append(mse_loss_arr,mse_loss.detach().cpu().numpy())

print(np.average(loss_arr))

In [ ]:
# Decode and Encode speed

compress_batch_size = 64
D = test_input_array.shape[1]

model = Network(64)

checkpoint = torch.load("compressai_losses_3.pth",map_location="cuda")

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

model.to("cuda")

print("Encode and Decode Time:")

total_encode_time_array = np.array([])
total_decode_time_array = np.array([])
total_encode_time = 0
total_decode_time = 0

compress_dataloader = DataLoader(test_input_array, batch_size=compress_batch_size, shuffle=False)
for i_batch, batch in enumerate(compress_dataloader):
        x = batch.to("cuda")
        x = x.to(torch.float)

        if i_batch == 156:
            x = x.reshape(16, 1, D)
        else:
            x = x.reshape(compress_batch_size, 1, D)

        start_time = time.time()
        y_hat, y_likelihoods = model.get_compressed_embeddings(x)
        end_time = time.time()
        total_encode_time += end_time - start_time

        start_time = time.time()
        x_hat = model.decode_compressed_embeddings(y_hat)
        end_time = time.time()
        total_decode_time += end_time - start_time

        total_encode_time_array = np.append(total_encode_time_array,total_encode_time)
        total_decode_time_array = np.append(total_decode_time_array,total_decode_time)

print("Avg encode time: " + str(np.average(total_encode_time_array)))
print("Avg decode time: " + str(np.average(total_decode_time_array)))